In [1]:
import pandas as pd
import numpy as np

import logging
import ast

from sentence_transformers import SentenceTransformer
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)

In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [4]:
EMBEDDING_MODEL =  'paraphrase-multilingual-mpnet-base-v2'

In [5]:
def preprocess_product_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess product dataset
    """
    df.fillna('', inplace=True)
    df['product_gender'] = df['product_gender'].apply(ast.literal_eval).apply(lambda x: ','.join(x)) 
    df['product_style'] = df['product_style'].apply(ast.literal_eval).apply(lambda x: ', '.join(x)) 
    df['product_note'] = df['product_note'].apply(ast.literal_eval).apply(lambda x: ', '.join(x)) 
    df['combined_text'] = (
        df['full_name'].str.strip() + ' ' +
        df['product_gender'].str.strip() + ' ' +
        df['brand'].str.strip() + ' ' +
        df['description'].str.strip() + ' ' +
        df['origin'].str.strip() + ' ' +
        df['product_style'].str.strip() + ' ' +
        df['product_note'].str.strip()
    )

    numeric_cols = df.drop(['parent_id'], axis=1).dtypes[(df.dtypes=='int64') | (df.dtypes=='float64')].index.values
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df[df['price'] >= 0]
    df = df[df['product_gender'] != '']
    df = df[df['description']!='']    
    return df

In [6]:
logger.info("Loading datasets...")
df = pd.read_csv('../data/namperfume_product.csv')
processed = preprocess_product_data(df)
print(processed.shape)
processed.head(1)

2025-03-18 21:59:00,847 - INFO - Loading datasets...


(1538, 25)


,id,name,full_name,parent_handle,sku,production_year,origin,price,compare_at_price,parent_id,total_quantity,sold_quantity,brand,product_gender,product_style,product_note,rate,count_rate,count_rate_1,count_rate_2,count_rate_3,count_rate_4,count_rate_5,description,combined_text
0,1003772321,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml Tester,dsquared2-icon-pour-homme,110100204700,2024,Ý,2200000,0,1003772317,0,0,DSQUARED2,Nam,"Tinh tế, Nam tính, Cuốn hút","Hương gừng, Cây xô thơm",0.0,0,0,0,0,0,0,"Hương đầu: Gừng, Cam chanh Hương giữa: Hoa tulip, Xô thơm, Phong lữ, Oải hương Hương cuối: Akiga...","Dsquared2 Icon Pour Homme - 100ml Tester Nam DSQUARED2 Hương đầu: Gừng, Cam chanh Hương giữa: Ho..."


In [7]:
def create_product_embeddings(df: pd.DataFrame, model_name: str = EMBEDDING_MODEL) -> np.ndarray:
    """
    Create embeddings for product descriptions
    """
    logger.info(f"Creating embeddings using {model_name}...")
    
    model = SentenceTransformer(model_name)
    texts = df.apply(
        lambda x: f"{x['combined_text']}", 
        axis=1
    ).tolist()
    embeddings = model.encode(texts,show_progress_bar=True,batch_size=32)
    
    return embeddings

In [8]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chroma_db_v1")
collection = chroma_client.get_or_create_collection(name="new_documents")

processed_list = processed.to_dict(orient="records")
embed = create_product_embeddings(processed).tolist()
for i, product in enumerate(processed_list):
    collection.add(
        ids=[str(product['id'])],  
        embeddings=[embed[i]],
        metadatas=[{
            "name": product["name"],
            "description": product["description"],
            "product_style": product['product_style'],
            "production_year": product["production_year"],
            'origin': product['origin'], 
            'price': product['price'], 
            'compare_at_price': product['compare_at_price'], 
            'sold_quantity': product['sold_quantity'],
            'brand': product['brand'],
            'product_gender': product['product_gender'], 
            'product_note': product['product_note'], 
            'rate': product['rate'], 
            'count_rate': product['count_rate'], 
            'count_rate_1': product['count_rate_1'], 
            'count_rate_2': product['count_rate_2'], 
            'count_rate_3': product['count_rate_3'],
            'count_rate_4': product['count_rate_4'],
            'count_rate_5': product['count_rate_5']
            
            }]
    )

print(f"Stored {collection.count()} documents in ChromaDB.")


2025-03-18 21:59:17,691 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


2025-03-18 21:59:17,868 - INFO - Creating embeddings using paraphrase-multilingual-mpnet-base-v2...
2025-03-18 21:59:17,910 - INFO - Use pytorch device_name: mps
2025-03-18 21:59:17,911 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
Batches: 100%|██████████| 49/49 [00:51<00:00,  1.06s/it]


Stored 1538 documents in ChromaDB.


In [9]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [16]:
def rag_query(question):
    model = SentenceTransformer(EMBEDDING_MODEL)
    query_embedding = model.encode([question], convert_to_tensor=True).tolist()

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3  
    )
    context = "\n\n".join(['\n'.join([res['name'],res['product_gender'], res["description"]]) for res in results["metadatas"][0]])
    print(f'=== context === \n {context}')
    prompt = f"Using the following context:\n{context}\nAnswer the question: {question}"
    response = genai.GenerativeModel("gemini-2.0-flash").generate_content(prompt)

    return response.text

Test query

In [17]:
answer = rag_query("nước hoa cho nam mùi hương nhẹ nhàng")
print("Generated Response:", answer)

2025-03-19 10:14:55,014 - INFO - Use pytorch device_name: mps
2025-03-19 10:14:55,016 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
Batches: 100%|██████████| 1/1 [00:00<00:00,  8.83it/s]


=== context === 
 Versace Pour Homme Dylan Blue
Nam
Hương đầu: Hương nước, Cam Bergamot Calabria, Quả bưởi, Lá sung Hương giữa: Lá hoa tím, Cây hoắc hương, Tiêu đen, Hương Ambroxan, Giấy cói Hương cuối: Xạ hương, Nhang (Hương), Đậu Tonka, Saffron Nếu như có ai hỏi tôi đã bao giờ bị một ai đó mê hoặc bởi mùi hương hay chưa, có lẽ tôi sẽ không do dự mà nói rằng đã từng, và trong đầu nghĩ ngay đến mùi hương man mát, nam tính được toả ra bởi Versace Pour Homme Dylan Blue. Tôi bị ấn tượng bởi những nốt hương của chai nước hoa này bởi sự mở đầu tinh khiết, trong trẻo của hương nước, Cam Bergamot và Bưởi. Nhưng trái với sự phóng khoáng, tươi mới ấy, Dylan Blue còn hấp dẫn tôi bởi cái sự nam tính của Hương Ambroxan, sự gai góc đầy cá tính của Tiêu đen. Chàng ta còn có thêm cái ấm áp, ngọt ngào của Xạ hương và Đậu tonka. Thứ hương đầy gợi cảm ấy như một cái bẫy ngọt ngào khiến bao con mồi tự sa vào. Bạn biết đó, tôi cũng là một trong số những con mồi bị thu hút bởi thứ hương sạch sẽ, cuốn hút ấ

In [ ]:
from FlagEmbedding import FlagReranker

In [25]:
query = "nước hoa cho nam mùi hương nhẹ nhàng"

In [ ]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="chroma_db_v1")
collection = chroma_client.get_or_create_collection(
    name="new_documents",
    embedding_function=SentenceTransformerEmbeddingFunction(model_name=EMBEDDING_MODEL)
)
model = SentenceTransformer(EMBEDDING_MODEL)
query_embedding = model.encode([query], convert_to_tensor=True).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=5 
)

metadatas = results["metadatas"]

# context = "\n\n".join(['\n'.join([res['name'],res['product_gender'], res["description"]]) for res in metadatas])

# Initialize BGE reranker
reranker = FlagReranker("BAAI/bge-reranker-large", use_fp16=True)
# Chuẩn bị input cho reranker
# pairs = [(query, [[doc['name'],doc['product_gender'],doc['product_style'],doc["description"]] for doc in metadatas[0]])]
# pairs = [(query, doc['name'] + ' ' + doc['product_gender'] + ' ' +doc['product_style'] + ' ' +doc["description"]) for doc in metadatas[0]]

# scores = reranker.compute_score(pairs)

# # Sắp xếp theo điểm rerank
# ranked_results = sorted(zip(metadatas, scores), key=lambda x: x[2], reverse=True)

# # Chọn top rerank_k tài liệu tốt nhất
# top_docs, top_metadatas, _ = zip(*ranked_results[:5])


2025-03-19 17:00:39,258 - INFO - Use pytorch device_name: mps
2025-03-19 17:00:39,259 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
Batches: 100%|██████████| 1/1 [00:00<00:00, 10.42it/s]


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
doc_tmp = [doc['name'] + ' ' + doc['product_gender'] + ' ' +doc['product_style'] + ' ' +doc["description"] for doc in metadatas[0]]
pairs = [(query, doc['name'] + ' ' + doc['product_gender'] + ' ' +doc['product_style'] + ' ' +doc["description"]) for doc in metadatas[0]]

scores = reranker.compute_score(pairs)

# Sắp xếp theo điểm rerank
ranked_results = sorted(zip(doc_tmp, scores), reverse=True)

# Chọn top rerank_k tài liệu tốt nhất
top_docs, top_metadatas = zip(*ranked_results[:5])


In [67]:
top_docs, top_metadatas

(('Viktor&Rolf Flowerbomb Dew Nữ Nữ tính, Gợi cảm, Dịu dàng Hương đầu: Lê, Ambrette, Giọt sương, Cam bergamot Hưng giữa: Hoa diên vĩ, Hoa hồng Hương cuối: Xạ hương, Len cashmeran, Hoa vòi voi Đôi khi ta không cần phải cá tính để được nổi bật, hay gợi cảm để được chú ý đến, là phụ nữ, bạn chỉ cần chọn cho mình một mùi hương bạn thích, đúng tâm trạng, giản đơn nhưng đủ khiến bản thân thấy vui. Hay quý giá hơn thảy là tự mình cảm được chất “thơ" trong mùi hương ấy mà không cần đến bất kỳ sự khen ngợi, ca tụng của ai. Tôi phải thừa nhận mình si mê Flowerbomb Dew Viktor & Rolf ngay từ những giây phút đầu tiên, vẻ lãng đãng, mộng mơ của cô nàng đã khiến tôi phải suy đoán trong rối bời những nốt hương tổng thể mà cô mang. Mãi đến sau này, tôi mới biết sự phiêu bồng ấy nhờ tông vị của Xạ hương cùng Len cashmeran cấu thành nên. Cùng với nét dịu dàng rất đỗi nên thơ của Hoa diên vĩ cộng hưởng cùng Hoa hồng, những ai ban đầu nói “không" thì chắc chắn sẽ rất hối hận khi Flowerbomb Dew Viktor & Rol